In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
training_size_arr_actual = [0, 0, 0]
training_size_arr_simulated = [0, 0, 0, 0]

dirNameInput = 'D:\Courses\CS286 ML\Project\Code\CAN Traffic Analyzer\CAN_Traffic_Analysis\Word2Vec\Word2VecData_Input'
dirNameOutput = 'D:\Courses\CS286 ML\Project\Code\CAN Traffic Analyzer\CAN_Traffic_Analysis\Word2Vec\Word2VecData_Output'


Word2Vec_model = None

X = None
Y = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None


D:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def createDataFiles():
    import time
    start = time.time()

    fileNamesActual = ['autopark.dat', 'drive.dat', 'idle.dat']
    fileInput = open("Word2VecData_Input\Word2VecDataActual.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataActual.txt", "w")
    stride = 2

    for i in range(len(fileNamesActual)):
        with open(fileNamesActual[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        value = int(0.8 * len(content))
    
        training_size_arr_actual[i] = value
        
        value = len(content)
        print(value)

        cntCntr = 0
        while cntCntr < value:
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < value:
                parts = content[cntCntr].split(",")

                # this is the whole message
                DATA = parts[3].split(":")[1].replace(" ", "")

                currMessagePackets += str(DATA) + " "

                cntCntr += 1
                iteration += 1

            if cntCntr < value:
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(i) + "\n")
    fileInput.close()
    fileOutput.close()

    fileInput = open("Word2VecData_Input\Word2VecDataSimulated.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['New_DRIVE_Data.txt', 'New_IDLE_Data.txt', 'DRIVE.rtf', 'IDLE.rtf']
    arrSimulated = [1, 2, 1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        
        training_size_arr_simulated[i] = (0.8 * len(content))
        
        print(len(content))

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < len(content):
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)


In [3]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=500, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [4]:
def train():
    createDataFiles()
    createWord2VecModels()



In [5]:
train()

5592
5582
5601
29977
29993
1814
1802
Awesome !!! File processing done !!!
Total Time for file processing --->  0.4500925540924072


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  2.9723165035247803


In [6]:
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
x_train = None
def getNumpyArray():
    global X
    global Y
    global dirNameInput
    X = None
    Y = None
    print("Processing started ")
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            currV = Word2Vec_model.wv[parts[0]]
            currV.setflags(write=1)
            for i in range(1, len(parts)):
                currV += Word2Vec_model.wv[parts[i]]
            currV /= len(parts)
            if X is None:
                X = currV
            else:
                X = np.vstack([X, currV])
                
    print("Processing started for output ")
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            if Y is None:
                Y = currV
            else:
                Y = np.vstack([Y, currV])
            
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)





In [14]:
def createDNNModel():
    global model 
    getNumpyArray()
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    

    

    

In [15]:
createDNNModel()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.30, random_state=1)


In [17]:
def executeModel(eph):
    start = time.time()
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=eph)
    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)



In [18]:
start = time.time()
executeModel(50)
end = time.time()
print("Total Time -->", end - start)

Epoch 1/50
18743/18743 [==============================] - 3s 154us/step - loss: 0.6804 - acc: 0.6482
Epoch 2/50
18743/18743 [==============================] - 2s 97us/step - loss: 0.3510 - acc: 0.8540
Epoch 3/50
18743/18743 [==============================] - 2s 98us/step - loss: 0.2675 - acc: 0.8977
Epoch 4/50
18743/18743 [==============================] - 2s 96us/step - loss: 0.2179 - acc: 0.9190
Epoch 5/50
18743/18743 [==============================] - 2s 99us/step - loss: 0.1990 - acc: 0.9241
Epoch 6/50
18743/18743 [==============================] - 2s 98us/step - loss: 0.1817 - acc: 0.9289
Epoch 7/50
18743/18743 [==============================] - 2s 100us/step - loss: 0.1718 - acc: 0.9322
Epoch 8/50
18743/18743 [==============================] - 2s 101us/step - loss: 0.1593 - acc: 0.9385
Epoch 9/50
18743/18743 [==============================] - 2s 100us/step - loss: 0.1475 - acc: 0.9418
Epoch 10/50
18743/18743 [==============================] - 2s 100us/step - loss: 0.1269 - acc: 0

In [19]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

8034/8034 [==============================] - 0s 48us/step
Test score: 0.017326491620924096
Test accuracy: 0.995394573064476


In [20]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()

Loaded successfully
500
500
<class 'numpy.ndarray'>
